## Using Pandas and Bokeh to Analyze and Visualze NYC Cooperative and Condominium Apartment Sales - PART 2

This is Part 2 of the tutorial, in Part 1 we cleaned and augmented the data into a dataframe that we saved as a DF. Now we are going to do the following: 

    4) Use "groupby" and "sort" to obtain the top 10 sales per building (by both number and 
       percentage)
    5) Merge the data with information on number of residential units per building from the PLUTO database so that 
    percentage sales per building can be calculated 
    6) Create a series of bar charts using Bokeh to visualize this data


In [330]:
import sqlite3
import pandas as pd
import pandas.io.sql as pd_sql
import numpy as np

# read in all rolling cleaned from Part 1
df=pd.read_csv('processed/RollingJulyProcessed.csv',na_values=[' ',''])

In [331]:
print(df.shape)
print (df.dtypes)
df.head(5)

(14833, 23)
Unnamed: 0                          int64
BOROUGH                             int64
NEIGHBORHOOD                       object
BUILDING CLASS CATEGORY            object
TAX CLASS AT PRESENT               object
BLOCK                               int64
LOT                                 int64
EASEMENT                          float64
BCLASS                             object
ADDRESS                            object
APT                                object
ZIP CODE                            int64
RESIDENTIAL UNITS                   int64
COMMERCIAL UNITS                    int64
TOTAL UNITS                         int64
LAND SQUARE FEET                    int64
GROSS SQUARE FEET                   int64
YEAR BUILT                          int64
TAX CLASS AT TIME OF SALE           int64
BUILDING CLASS AT TIME OF SALE     object
SALE PRICE                          int64
SALE DATE                          object
CONDOLOT                            int64
dtype: object


,Unnamed: 0,BOROUGH,NEIGHBORHOOD,BUILDING CLASS CATEGORY,TAX CLASS AT PRESENT,BLOCK,LOT,EASEMENT,BCLASS,ADDRESS,...,COMMERCIAL UNITS,TOTAL UNITS,LAND SQUARE FEET,GROSS SQUARE FEET,YEAR BUILT,TAX CLASS AT TIME OF SALE,BUILDING CLASS AT TIME OF SALE,SALE PRICE,SALE DATE,CONDOLOT
0,85,1,ALPHABET CITY,13 CONDOS - ELEVATOR APARTMENTS,2,373,7501,NaN,R4,324 EAST 4TH STREET,...,0,1,0,0,1920,2,R4,1395796,2016-07-28,1002
1,86,1,ALPHABET CITY,13 CONDOS - ELEVATOR APARTMENTS,2,373,7501,NaN,R4,324 EAST 4TH STREET,...,0,1,0,0,1920,2,R4,792812,2016-07-28,1010
2,87,1,ALPHABET CITY,13 CONDOS - ELEVATOR APARTMENTS,2,375,7501,NaN,R4,754 EAST 6TH STREET,...,0,1,0,0,0,2,R4,975000,2015-08-14,1017
3,88,1,ALPHABET CITY,13 CONDOS - ELEVATOR APARTMENTS,2,375,7501,NaN,R4,754 EAST 6TH STREET,...,0,1,0,0,0,2,R4,840000,2015-12-30,1023
4,89,1,ALPHABET CITY,13 CONDOS - ELEVATOR APARTMENTS,2,384,7503,NaN,R4,310 EAST HOUSTON STREET,...,0,1,0,0,2005,2,R4,0,2016-04-25,1224


Now to calculate sales by building and sales by unit using groupby.  Once I create a groupby function I then execute a series of calculations on it. Since we are concerned with the number of occurences of each Block / Lot combination we could of used and column to calculate the length of.  I choose ['Sale Price'] because it could be interesting for future analysis.

In [332]:
# calculate data on sales by building
bybuilding = df.groupby(['BLOCK','LOT'])
salesbybuilding = bybuilding['SALE PRICE'].agg([np.sum,np.mean,np.std,len])
salesbybuilding=salesbybuilding.rename(columns={'len':'NUMSALES'})
salesbybuilding=salesbybuilding.reset_index()

# calculate data on sales by unit
byapt = df.groupby(['BLOCK','LOT','APT'])
salesbyapt = byapt['SALE PRICE'].agg([np.sum,np.mean,np.std,len])
salesbyapt=salesbyapt.rename(columns={'len':'NUMSALES'})
salesbyapt=salesbyapt.reset_index()


PLUTO database has nearly 50 columns of information on every property in New York City.  I have previously downloaded the data to a database and will use this to create a dataframe to add owner, # of residential units and class of property to the processed dataframe of rolling data.

In [333]:
def getpluto():
    db = sqlite3.connect('processed/NYCPROPERTY.db')
    cursor = db.cursor()
    cursor.execute('SELECT BLOCK, LOT, OwnerName, Unitsres, BldgClass FROM condoandcoop')
    result = cursor.fetchall()
    df_coop = pd.DataFrame(result, columns=['BLOCK','LOT', 'OWNER', 'UNITSRES','CLASS'])
    cursor.close()
    df_coop =df_coop.sort_values('BLOCK').drop_duplicates(subset=['BLOCK', 'LOT'], keep='last')
    return df_coop

plutodata=getpluto()

As shown below the plutodata uses block and lot to idnetify the buildings (similar to the df that we have been working on).  So I will use this to merge the data.  

In [334]:
print(plutodata.shape)
print (plutodata.dtypes)
plutodata.head(5)

(6157, 5)
BLOCK        int64
LOT          int64
OWNER       object
UNITSRES     int64
CLASS       object
dtype: object


,BLOCK,LOT,OWNER,UNITSRES,CLASS
0,4,7501,ONE NY PLAZA CO LLC,0,RC
1,5,7501,125 BROAD CONDOMINIUM,0,RB
2,7,7501,66 PEARL STREET HOLDI,42,RM
3,7,7502,None,1,RM
4,8,7501,NEW YORK PLAZA RESIDE,97,RM


Now that we have created the groupby we can add the information from the Pluto database.

In [335]:
def fixapt(ldf):  
    if len(ldf['APT'].strip())<1:
        ldf['APT'] = ldf['APT2'] 
    return(ldf['APT'])   
        
def stripname(ldf):
    x=ldf['ADDRESS']    
    try:
        x=x.strip(' ')
        x=x.replace("STREET","ST")  
        x=x.replace("EAST","E")
        x=x.replace("WEST","W")
        x=x.replace("AVENUE","AVE")
        x=x.replace("BOULEVARD","BLVD")
        x=x.replace("SOUTH","S")             
    except:
        x=x
    return(x)
    
# Add unitsres and owner
salesbybuilding = pd.merge(salesbybuilding, plutodata, on=['BLOCK','LOT'], how='left')
salesbyapt = pd.merge(salesbyapt, plutodata, on=['BLOCK','LOT'], how='left')

# Add address and apartment information
address_df = df[['BLOCK', 'LOT','CONDOLOT','ADDRESS','BCLASS']]
address_df = address_df.sort_values('BLOCK').drop_duplicates(subset=['BLOCK', 'LOT'], keep='last')
salesbybuilding = pd.merge(salesbybuilding, address_df, on=['BLOCK','LOT'], how='left')
salesbybuilding['ADDRESS']=salesbybuilding.apply(stripname, axis=1)

# top 20 sales by number
mostsales =salesbybuilding.sort_values('NUMSALES',ascending=False).head(20)
salescoop = salesbybuilding[salesbybuilding['CONDOLOT'] <1]
mostsalescoop = salescoop.sort_values('NUMSALES',ascending=False).head(20)

In [336]:
salesbybuilding['UNITSRES'].isnull().value_counts()

False    2911
True       26
Name: UNITSRES, dtype: int64

In [337]:
salescoop.dtypes

BLOCK         int64
LOT           int64
sum           int64
mean          int64
std         float64
NUMSALES      int64
OWNER        object
UNITSRES    float64
CLASS        object
CONDOLOT      int64
ADDRESS      object
BCLASS       object
dtype: object

In [338]:
totalsales = salesbybuilding['NUMSALES'].sum()
print(totalsales)

14833


Now to calculate as a percentage.

In [339]:
def getpercent(ldf):
    try:
        ldf['PERSALES'] = ldf['NUMSALES'] / ldf['UNITSRES']
    except:     
        ldf['PERSALES'] = float('nan')
    return ldf['PERSALES']

# Calculate the percentage sold and add column with info
salesbybuilding['PERSALES']=salesbybuilding.apply(getpercent, axis=1)

salesbybuilding[200:210]

,BLOCK,LOT,sum,mean,std,NUMSALES,OWNER,UNITSRES,CLASS,CONDOLOT,ADDRESS,BCLASS,PERSALES
200,389,20,450000,450000,NaN,1,TOMFOL OWNERS CORP.,24.0,C6,0,204 E 7 ST,C6,0.041667
201,389,21,90000,90000,NaN,1,TOMFOL RLTY CORP,24.0,C6,0,206 E 7 ST,C6,0.041667
202,389,48,675000,675000,NaN,1,625 EAST 6TH ST HOUSI,10.0,C6,0,625 E 6TH ST,C6,0.100000
203,389,50,400000,400000,NaN,1,621-23 EAST 6TH HDFC,10.0,C6,0,621 E 6TH ST,C6,0.100000
204,389,54,855000,855000,NaN,1,613 EAST 6TH ST OWNER,14.0,C6,0,613 E 6TH ST,C6,0.071429
205,390,42,475000,475000,NaN,1,219 E 7TH ST HOUSING,24.0,C6,0,219 E 7TH ST,C6,0.041667
206,391,3,980000,980000,NaN,1,TOMPKINS PK MANSION O,18.0,C6,0,131 AVE B,C6,0.055556
207,391,20,715872,238624,356580.403455,3,626 E 9 ST HDFC,12.0,C6,0,626 E 9TH ST,C6,0.250000
208,391,48,820000,820000,NaN,1,323 EAST 8TH STREET O,25.0,C6,0,323 E 8TH ST,C6,0.040000
209,392,36,630000,630000,NaN,1,HOME AT LAST CORP,9.0,C6,0,149 AVE C,C6,0.111111


In [340]:
salesbybuilding.shape

(2937, 13)

Going to use Bokeh library to visualize the data.

In [341]:
from bokeh.plotting import figure, output_file, show, output_notebook
from bokeh.charts import Bar, hplot, Scatter
from bokeh.models import HoverTool
from bokeh.layouts import row, column, gridplot
import pandas.io.sql as pd_sql
from bokeh.charts.attributes import CatAttr

In [342]:
output_notebook()

hover = HoverTool(
        tooltips=[("Building","@numsales")])
      
hover.tooltips = [('Value of ID',' $x'),('Value of Total',' @y')]
        
bar_numsales = Bar(mostsales, values='NUMSALES', label=CatAttr(columns=['ADDRESS'], sort=False), color='green', ylabel='Sales in the last 12 Months', title='Most Unit Sales Coops & Condos',legend=None,tools='hover')
#bar_numcoops = Bar(mostsalescoop, 'ADDRESS', values='numsales',color='green', ylabel='Sales in the last 12 Months', title='Most Sales by Coop Building',legend=None)

bar_numcoops = Bar(mostsalescoop, values='NUMSALES', color='green', ylabel='Sales in the last 12 Months', label=CatAttr(columns=['ADDRESS'], sort=False),title='Most Unit Sales Coop Buildings',legend=None,tools='hover')

# top 20 sales by percentage
largestper =salesbybuilding.sort_values('PERSALES',ascending=False).head(20)
largestper['ADDRESS']=largestper.apply(stripname, axis=1)
bar_persales = Bar(largestper, values='PERSALES', label=CatAttr(columns=['ADDRESS'], sort=False), color='green', ylabel='Sales in the last 12 Months', title='Greatest % Unit Sales in Coop & Condo Buildings',legend=None, tools='hover')

# top 20 sales coops by percentage
salescoop = salesbybuilding[salesbybuilding['CONDOLOT'] <1]
largest_coop_per = salescoop.sort_values('PERSALES',ascending=False).head(20)
bar_coop_persales = Bar(largest_coop_per, values='PERSALES', label=CatAttr(columns=['ADDRESS'], sort=False), color='green', ylabel='Sales in the last 12 Months', title='Greatest % Unit Sales in Coop Buildings',legend=None)

# make a grid
grid = gridplot([bar_numsales, bar_numcoops, bar_persales, bar_coop_persales], ncols=2, plot_width=400, plot_height=400)
show(grid)


Loading BokehJS ...